In [81]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [82]:
N_EPOCHS = 200
N_TRIALS = 3
BATCH_SIZE = 256

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [83]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
x, y = utils.load_data("../data/cifar-10-batches-py/data_batch_1")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

train_size = x_train.shape[0]

In [84]:
x_train = np.clip(x_train + np.random.normal(0, 20, x_train.shape), 0, 255)
x_val = np.clip(x_val + np.random.normal(0, 20, x_val.shape), 0, 255)
x_test = np.clip(x_test + np.random.normal(0, 20, x_test.shape), 0, 255)

In [85]:
results = {}

# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]

In [86]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=10, **models_hyperparameters[test_model])
        
        model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
        
        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(2 * (i + 1)/ N_EPOCHS) * train_size)
            samples_ids = np.random.choice(range(train_size), n_samples, replace=False)

            model.fit(
                x_train[samples_ids], y_train[samples_ids], 
                validation_data=(x_val, y_val),
                epochs=1, batch_size=BATCH_SIZE,
                verbose=1, callbacks=[early_stopping]
            )
        
        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)
        
        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/3 [00:00<?, ?it/s]

 1/25 [>.............................] - ETA: 1s - loss: 1.0964 - accuracy: 0.5820

In [80]:
results

{'test_model_1': [0.49142858386039734, 0.5022857189178467, 0.5157142877578735]}